In [ ]:
!apt-get install tree

In [ ]:
!maskrcnn-modanet datasets download ./tmp
!cp -r tmp/datasets/coco/images ./
!rm -rf tmp

In [ ]:
import cv2
import face_recognition
import torch
import os
from tqdm.notebook import tqdm

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='Weights/best.pt')
model.conf = 0.5

In [ ]:
classes = ['bag', 'belt', 'boots', 'footwear', 'outer', 'dress', 'sunglasses', 'pants', 'top', 'shorts', 'skirt', 'headwear', 'scarf/tie']

In [ ]:
!mkdir preprocessed_dataset
count = 0
for image_path in tqdm(os.listdir('images')):
    path = f'images/{image_path}'
    imgs =[path]
    result = model(imgs)
    clothes = result.xyxy[0].cpu().numpy().astype(int)

    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_location = face_recognition.face_locations(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    clothes_images = [image[pred[1]:pred[3], pred[0]:pred[2]] for pred in clothes]

    if len(face_location) == 1 and len(clothes_images) > 0:
        count += 1
        os.makedirs(f'preprocessed_dataset/{count}', exist_ok=True)
        face_image = image[face_location[0][0]:face_location[0][2],face_location[0][3]:face_location[0][1]]
        cv2.imwrite(f'preprocessed_dataset/{count}/face.jpg', face_image)
        for i in range(len(clothes_images)):
            cv2.imwrite(f'preprocessed_dataset/{count}/{i}.jpg', clothes_images[i])
!rm -rf images